In [ ]:
import os
import pandas as pd

In [ ]:
path = './dataset'
datasets_path = path+'/dataset/solar_stations'

In [ ]:

files = [os.path.join(datasets_path, file) for file in os.listdir(datasets_path) if file.endswith('.xlsx') and '_3_' not in file]
files.sort()

df_list = []
for file in files:
  print(file)
  df = pd.read_excel(file)
  df_list.append(df)

print(len(df_list))

for i in range(0,len(df_list)):
  print(files[i])
  print(df_list[i].isnull().sum())
  print(df_list[i].isna().sum())

df_list[2][df_list[2].isna().any(axis=1)]
df_list[2].interpolate(method='linear', inplace=True)

df_list[2].isna().sum()

/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 1 (Nominal capacity-50MW).xlsx
/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 2 (Nominal capacity-130MW).xlsx
/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 3 (Nominal capacity-30MW).xlsx
/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 4 (Nominal capacity-130MW).xlsx
/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 5 (Nominal capacity-110MW).xlsx
/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 6 (Nominal capacity-35MW).xlsx
/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 7 (Nominal capacity-30MW).xlsx
/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 8 (Nominal capacity-30MW).xlsx
8
/home/yglee/workspace/reforecast/dataset/solar_stations/Solar station site 1 (Nominal capacity-50MW).xlsx
Time(year-month-day h:m:s)             0


Time(year-month-day h:m:s)             0
Total solar irradiance (W/m2)          0
Direct normal irradiance (W/m2)        0
Global horizontal irradiance (W/m2)    0
Atmosphere (hpa)                       0
Relative humidity (%)                  0
Power (MW)                             0
dtype: int64

In [ ]:
import numpy as np
import math

for df in df_list:
  df['Time'] = pd.to_datetime(df['Time(year-month-day h:m:s)'])

  # 원하는 칼럼 추출
  df['month'] = df['Time'].dt.month
  df['day'] = df['Time'].dt.dayofweek
  df['hour'] = df['Time'].dt.hour

  df['day_sin'] = np.sin(df['day']*2*math.pi/7)
  df['day_cos'] = np.cos(df['day']*2*math.pi/7)
  df['hour_sin'] = np.sin(df['hour']*2*math.pi/24)
  df['hour_cos'] = np.cos(df['hour']*2*math.pi/24)
  df['month_sin'] = np.sin(df['month']*2*math.pi/12)
  df['month_cos'] = np.cos(df['month']*2*math.pi/12)

  df.sort_values(by='Time', inplace=True)


In [ ]:
import pandas as pd
from datetime import timedelta

feature_added_df_list = []

for df in df_list:
    df['Time'] = pd.to_datetime(df['Time'])

    def calc_stats(row):
        end = row['Time']
        start = end - timedelta(days=7)
        window = df[(df['Time'] >= start) & (df['Time'] < end)]

        # 전체 발전량
        pow_all = window['Power (MW)']
        # 같은 시간대 발전량
        pow_same_time = window[window['Time'].dt.time == end.time()]['Power (MW)']
        # 같은 요일 발전량
        pow_same_weekday = window[window['Time'].dt.weekday == end.weekday()]['Power (MW)']

        return pd.Series({
            'same_time_mean':       pow_same_time.mean(),
            'same_time_std':        pow_same_time.std(),
            'same_time_max':        pow_same_time.max(),
            'same_time_min':        pow_same_time.min(),
            'same_weekday_mean':    pow_same_weekday.mean(),
            'same_weekday_std':     pow_same_weekday.std(),
            'same_weekday_max':     pow_same_weekday.max(),
            'same_weekday_min':     pow_same_weekday.min(),
            'overall_mean':         pow_all.mean(),
            'overall_std':          pow_all.std(),
            'overall_max':          pow_all.max(),
            'overall_min':          pow_all.min(),
        })

    stats = df.apply(calc_stats, axis=1)
    for col in stats.columns:
        df[col] = stats[col]

    feature_added_df_list.append(df)


In [ ]:
feature_added_df_list[0].columns

Index(['Time(year-month-day h:m:s)', 'Total solar irradiance (W/m2)',
       'Direct normal irradiance (W/m2)',
       'Global horizontal irradiance (W/m2)', 'Air temperature  (°C) ',
       'Atmosphere (hpa)', 'Power (MW)', 'Time', 'month', 'day', 'hour',
       'day_sin', 'day_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
       'same_time_mean', 'same_time_std', 'same_time_max', 'same_time_min',
       'same_weekday_mean', 'same_weekday_std', 'same_weekday_max',
       'same_weekday_min', 'overall_mean', 'overall_std', 'overall_max',
       'overall_min'],
      dtype='object')

In [ ]:
for file,df in zip(files,feature_added_df_list):
  file_name = file.split('/')[-1].replace('.xlsx','')
  df.to_csv(path+'/lag_added_dataset/feature_added_dataset_'+file_name+'.csv')

In [ ]:
lag_added_df_list = []
filtered_feature_added_df_list = [feature_added_df[feature_added_df['Time'] >= '2019-01-08'] for feature_added_df in feature_added_df_list]

for i in range(len(filtered_feature_added_df_list)):
    filtered_feature_added_df = filtered_feature_added_df_list[i]
    print("-------------------------------------------------------------")

    for lag in [1, 2, 3, 4, 5,6,7,8]:  # 15분 간격 x 1~5 step
      filtered_feature_added_df[f'lag_{lag}'] = filtered_feature_added_df['Power (MW)'].shift(lag)

    filtered_feature_added_df['diff_15min'] = filtered_feature_added_df['Power (MW)'] - filtered_feature_added_df['Power (MW)'].shift(1)
    filtered_feature_added_df['diff_30min'] = filtered_feature_added_df['Power (MW)'] - filtered_feature_added_df['Power (MW)'].shift(2)
    filtered_feature_added_df['diff_45min'] = filtered_feature_added_df['Power (MW)'] - filtered_feature_added_df['Power (MW)'].shift(3)
    filtered_feature_added_df['diff_60min'] = filtered_feature_added_df['Power (MW)'] - filtered_feature_added_df['Power (MW)'].shift(4)
    filtered_feature_added_df['diff_75min'] = filtered_feature_added_df['Power (MW)'] - filtered_feature_added_df['Power (MW)'].shift(5)
    filtered_feature_added_df['diff_90min'] = filtered_feature_added_df['Power (MW)'] - filtered_feature_added_df['Power (MW)'].shift(6)
    filtered_feature_added_df['diff_105min'] = filtered_feature_added_df['Power (MW)'] - filtered_feature_added_df['Power (MW)'].shift(7)

    filtered_feature_added_df.dropna(inplace=True)
    lag_added_df_list.append(filtered_feature_added_df)

In [ ]:
for file,df in zip(files,lag_added_df_list):
  file_name = file.split('/')[-1].replace('.csv','')
  print(file_name)
  df.to_csv(path+'/lag_added_dataset/'+file_name)